<a href="https://colab.research.google.com/github/Shibu4064/EMNLP/blob/main/sagor_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install transformers datasets
!pip install transformers
!pip install transformers[torch]
!pip install transformers[tf-cpu]
!pip install transformers[flax]
!pip install torch
!pip install tensorflow
!pip install evaluate

In [3]:
#import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/EMNLP/train.csv') # train set
val_df = pd.read_csv('/content/drive/MyDrive/EMNLP/dev.csv') # validation set

In [4]:
print(len(train_df), len(val_df))

2700 1330


In [5]:
train_df.columns

Index(['text', 'label'], dtype='object')

In [6]:
train_df

,text,label
0,যে দেশে সন্ত্রাসরা দেশ চালায়সে দেশে শান্তি কিভ...,1
1,এই বিচার শেষ বিচার নয়।আসল বিচার হবে আল্লাহর আদ...,0
2,আরব দেশগুলোকে বলব ভারতের সাথে সব ব্যবসা বাণিজ্...,2
3,দেশটা সুস্থ নাই,0
4,আপনার কথা দুঃখ জনক আগে বিডিও থাকলে কেন ধরা হলন...,0
...,...,...
2695,"হল বন্ধ করার আগে নিউমার্কেট বন্ধ করা উচিৎ,,।",2
2696,এ স্বাধীন দেশে ভোটটা অন্ততঃ আমাদের দিতে দেন।জা...,0
2697,আল হামদুলিল্লাহ্...প্রিয় ভিউয়ার্স আপনাদের ভালব...,0
2698,এই গুলা ত বিচার করবায় ঐ তোমরা ত ইন্ডিয়ান না,0


In [7]:
from sklearn.model_selection import train_test_split
def pandaToList(dataset):
    text_list = []
    label_list = []
    for i in range(len(dataset['text'])):
      text_list.append(dataset['text'][i])
      label_list.append(int(dataset['label'][i]))
    return text_list, label_list

train_texts, train_labels =  pandaToList(train_df)
val_texts, val_labels = pandaToList(val_df)

In [8]:
!pip uninstall jaxlib

Found existing installation: jaxlib 0.4.13
Uninstalling jaxlib-0.4.13:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/jaxlib-0.4.13.dist-info/*
    /usr/local/lib/python3.10/dist-packages/jaxlib/*
Proceed (Y/n)? y
  Successfully uninstalled jaxlib-0.4.13


In [9]:
!pip install -U jax jaxlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 MB 9.8 MB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.4.14-py3-none-any.whl size=1535362 sha256=18a97bb3a5aff502c828c7df893661b7a7e9c2cf29a22eb6262b8b2ef2ec9eb2
  Stored in directory: /root/.cache/pip/wheels/85/52/e7/dfa571c9f9b879e3facaa1584f52be04c4c3d1e14054ef40ab
Successfully built jax
  Attempting uninstall: jax
    Found existing installation: jax 0.4.13
    Uninstalling jax-0.4.13:
      Successfully uninstalled jax-0.4.13


In [17]:
#tokenizer_name = 'bert-base-multilingual-cased'
#tokenizer_name='xlm-roberta-base'
tokenizer_name="sagorsarker/bangla-bert-base"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)

In [18]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [19]:
import torch

class ViolenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ViolenceDataset(train_encodings, train_labels)
val_dataset = ViolenceDataset(val_encodings, val_labels)


In [20]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [21]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, IntervalStrategy, AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import BertTokenizer, BertModel
training_args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy = IntervalStrategy.STEPS, # "steps"
   #output_dir= output_dir,
   eval_steps = 250, # Evaluation and Save happens every 250 steps
   save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.

   learning_rate=1e-5,
   per_device_train_batch_size=8,
   per_device_eval_batch_size=8,
   num_train_epochs=50,
   weight_decay=0.01,
   metric_for_best_model = 'f1',
   load_best_model_at_end=True)

model_name = "sagorsarker/bangla-bert-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3) ## change here for using another pretrained model

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(102025, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [23]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name} is trainable")
    else:
        print(f"{name} is non-trainable")

bert.embeddings.word_embeddings.weight is trainable
bert.embeddings.position_embeddings.weight is trainable
bert.embeddings.token_type_embeddings.weight is trainable
bert.embeddings.LayerNorm.weight is trainable
bert.embeddings.LayerNorm.bias is trainable
bert.encoder.layer.0.attention.self.query.weight is trainable
bert.encoder.layer.0.attention.self.query.bias is trainable
bert.encoder.layer.0.attention.self.key.weight is trainable
bert.encoder.layer.0.attention.self.key.bias is trainable
bert.encoder.layer.0.attention.self.value.weight is trainable
bert.encoder.layer.0.attention.self.value.bias is trainable
bert.encoder.layer.0.attention.output.dense.weight is trainable
bert.encoder.layer.0.attention.output.dense.bias is trainable
bert.encoder.layer.0.attention.output.LayerNorm.weight is trainable
bert.encoder.layer.0.attention.output.LayerNorm.bias is trainable
bert.encoder.layer.0.intermediate.dense.weight is trainable
bert.encoder.layer.0.intermediate.dense.bias is trainable
bert

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
250,No log,0.747023,0.666165,0.665769,0.556013,0.566886


In [ ]:
trainer.predict(val_dataset)

PredictionOutput(predictions=array([[ 5.2332745, -2.758482 , -2.2070894],
       [ 5.4174147, -2.8615146, -2.385967 ],
       [ 5.3723903, -3.0246358, -2.0558925],
       ...,
       [ 5.3098364, -2.8952785, -2.1806617],
       [-3.1345732,  5.354461 , -2.19704  ],
       [ 4.1970615, -1.8207769, -2.1045752]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 1, 0]), metrics={'test_loss': 1.453665018081665, 'test_accuracy': 0.7654135338345864, 'test_precision': 0.764505498627868, 'test_recall': 0.7039584454031856, 'test_f1': 0.7266556809767039, 'test_runtime': 12.8471, 'test_samples_per_second': 103.525, 'test_steps_per_second': 12.999})

In [ ]:
output_dir = "/content/drive/MyDrive/VITD-Models/xlm-roberta"

In [ ]:
trainer.save_model(f"{output_dir}") # saving the model to drive.